# OceanPulse: Sea Surface Temperature (SST) Visualization & Anomaly Detection
Welcome to OceanPulse — a combined exploration of real-time SST data (from MODIS Aqua CSV) and long-term SST climatology (from NOAA NetCDF).
This notebook visualizes ocean temperature and detects anomalies for April 2025.

In [ ]:
!pip install xarray netCDF4 pandas matplotlib seaborn

In [ ]:
# Upload both required files: CSV and NetCDF
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load and clean the CSV snapshot (April 2025)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

csv_df = pd.read_csv('MYD28M_2025-04-01_gs_250x125.SS.CSV')
csv_long = csv_df.melt(id_vars=['lat/lon'], var_name='longitude', value_name='sst')
csv_long.rename(columns={'lat/lon': 'latitude'}, inplace=True)
csv_long['latitude'] = pd.to_numeric(csv_long['latitude'])
csv_long['longitude'] = pd.to_numeric(csv_long['longitude'])
csv_long = csv_long[csv_long['sst'] < 1000]

In [ ]:
# Plot SST heatmap for April 2025
pivot_csv = csv_long.pivot(index='latitude', columns='longitude', values='sst').sort_index(ascending=False)
plt.figure(figsize=(14, 8))
sns.heatmap(pivot_csv, cmap='coolwarm', cbar_kws={'label': 'SST (°C)'})
plt.title('Snapshot SST — April 2025 (MODIS Aqua)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
import xarray as xr

ds = xr.open_dataset('sst.mnmean.nc')
df_nc = ds['sst'].to_dataframe().reset_index().dropna()

# Filter only April data
df_april = df_nc[df_nc['time'].dt.month == 4]
df_april_clim = df_april.groupby(['lat', 'lon'])['sst'].mean().reset_index()
df_april_clim.rename(columns={'sst': 'sst_climatology'}, inplace=True)


In [ ]:
from scipy.spatial import cKDTree
import numpy as np

# Prepare arrays
snapshot_coords = csv_long[['latitude', 'longitude']].values
climatology_coords = df_april_clim[['lat', 'lon']].values

# KDTree for nearest neighbor search
tree = cKDTree(climatology_coords)
dist, idx = tree.query(snapshot_coords, k=1)

# Get matched climatology points
matched_clim = df_april_clim.iloc[idx].reset_index(drop=True)
snapshot_clean = csv_long.reset_index(drop=True)

# Merge values
merged_nn = pd.concat([snapshot_clean, matched_clim], axis=1)
merged_nn['sst_anomaly'] = merged_nn['sst'] - merged_nn['sst_climatology']

In [ ]:
pivot_anomaly = merged_nn.pivot(index='latitude', columns='longitude', values='sst_anomaly').sort_index(ascending=False)

if not pivot_anomaly.empty and pivot_anomaly.notna().values.any():
    plt.figure(figsize=(14, 8))
    sns.heatmap(pivot_anomaly, cmap='RdBu_r', center=0, cbar_kws={'label': 'SST Anomaly (°C)'})
    plt.title('SST Anomaly — April 2025 vs Climatological April Average')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()
else:
    print("Still no data to plot — even after nearest neighbor fix.")

### Summary
- Visualized real SST data from MODIS Aqua for April 2025
- Compared it against 40+ years of NOAA SST records
- Computed and visualized temperature anomalies

_You can now extend this project by animating SST trends, exploring El Niño zones_